In [ ]:
# ===================== 全球CO₂排放分析 + 交互式仪表盘（最终完整版） =====================
# 1. 导入所有依赖库
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import warnings
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
import dash
from dash import dcc, html
import plotly.express as px

# 2. 基础配置（屏蔽警告+中文字体+路径创建）
warnings.filterwarnings("ignore")
plt.rcParams["font.family"] = ["Microsoft YaHei"]
plt.rcParams["axes.unicode_minus"] = False

# 路径配置（自动创建results文件夹）
CURRENT_DIR = os.getcwd()
DATA_DIR = os.path.join(CURRENT_DIR, "data")
RESULT_DIR = os.path.join(CURRENT_DIR, "results")
os.makedirs(RESULT_DIR, exist_ok=True)

# 步骤1：数据加载与预处理 
try:
    # 加载数据（确保data文件夹下有final_cleaned_data.csv）
    final_df = pd.read_csv(os.path.join(DATA_DIR, "final_cleaned_data.csv"))
    # 筛选2010-2020年数据 + 量级修正（匹配现实吨/人）
    model_df = final_df[final_df["Year"].between(2010, 2020)].copy()
    model_df["Per_Capita_GHG"] = model_df["Per_Capita_GHG"] * 1e6  # 校准排放单位
    model_df["Per_Capita_Energy"] = model_df["Per_Capita_Energy"] * 1e6  # 校准能源单位
    # 筛选核心特征 + 过滤异常值（0-50吨/人）
    model_df = model_df[["Country", "Year", "Per_Capita_GDP", "Per_Capita_Energy", "Per_Capita_GHG"]].dropna()
    model_df = model_df[(model_df["Per_Capita_GHG"] > 0) & (model_df["Per_Capita_GHG"] < 50)]
    print(f"✅ 数据加载完成 | 有效样本量：{model_df.shape[0]} | 人均排放范围：{model_df['Per_Capita_GHG'].min():.1f}~{model_df['Per_Capita_GHG'].max():.1f}吨/人")
except FileNotFoundError:
    print("❌ 错误：data文件夹下未找到final_cleaned_data.csv文件，请检查路径！")
    exit()

# 排放影响因素回归分析 
# 标准化特征 + 构建回归模型
scaler = StandardScaler()
X = scaler.fit_transform(model_df[["Per_Capita_GDP", "Per_Capita_Energy"]])
y = model_df["Per_Capita_GHG"].values
lr_model = LinearRegression()
lr_model.fit(X, y)

# 输出权重结果
feature_names = ["人均GDP", "人均能源消耗"]
coef_df = pd.DataFrame({
    "影响因素": feature_names,
    "权重系数": lr_model.coef_,
    "绝对权重": np.abs(lr_model.coef_)
}).sort_values("绝对权重", ascending=False)

print("\n===== 人均CO₂排放影响因素权重 =====")
print(coef_df.round(2))
print(f"模型解释力（R²）：{r2_score(y, lr_model.predict(X)):.3f}")

# 可视化权重并保存
plt.figure(figsize=(7, 5), dpi=150)
colors = ["#2A9D8F" if c > 0 else "#E76F51" for c in coef_df["权重系数"]]
bars = plt.bar(coef_df["影响因素"], coef_df["权重系数"], color=colors, width=0.6)
# 标注数值
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + (1 if height > 0 else -1), 
             f"{height:.1f}", ha="center", fontsize=11)
plt.title("人均CO₂排放影响因素权重（2010-2020）", fontsize=12)
plt.ylabel("权重系数（吨/人）", fontsize=11)
plt.axhline(y=0, color="#666", linestyle="--")
plt.grid(axis="y", alpha=0.2)
plt.tight_layout()
# 保存图片
weight_path = os.path.join(RESULT_DIR, "impact_factors_final.png")
plt.savefig(weight_path, dpi=150, facecolor="white")
plt.close()
print(f"✅ 权重图已保存 → {weight_path}")

# ===================== 步骤3：巴黎协定减排效果评估 =====================
def evaluate_paris_agreement():
    # 拆分协定前后数据
    pre_agreement = model_df[model_df["Year"].between(2010, 2015)].groupby("Year")["Per_Capita_GHG"].mean()
    post_agreement = model_df[model_df["Year"].between(2016, 2020)].groupby("Year")["Per_Capita_GHG"].mean()
    
    # 计算年均变化率
    pre_rate = (pre_agreement.iloc[-1] - pre_agreement.iloc[0]) / 5 / pre_agreement.iloc[0] * 100
    post_rate = (post_agreement.iloc[-1] - post_agreement.iloc[0]) / 5 / post_agreement.iloc[0] * 100
    
    print(f"\n===== 巴黎协定减排效果 =====")
    print(f"2010-2015（协定前）：年均变化率 {pre_rate:.1f}%")
    print(f"2016-2020（协定后）：年均变化率 {post_rate:.1f}%")
    print(f"减排速率提升倍数：{abs(post_rate/pre_rate):.1f}倍")

    # 可视化趋势对比
    plt.figure(figsize=(9, 5), dpi=150)
    plt.plot(pre_agreement.index, pre_agreement.values, 
             color="#E76F51", linewidth=3, marker="o", label=f"协定前（{pre_rate:.1f}%/年）")
    plt.plot(post_agreement.index, post_agreement.values, 
             color="#2A9D8F", linewidth=3, marker="s", label=f"协定后（{post_rate:.1f}%/年）")
    # 标注2015年协定签署
    plt.axvline(x=2015, color="#666", linestyle="--", label="巴黎协定签署（2015）")
    plt.title("2010-2020 巴黎协定前后人均CO₂排放变化", fontsize=12)
    plt.xlabel("年份", fontsize=11)
    plt.ylabel("人均CO₂排放（吨/人）", fontsize=11)
    plt.xticks(list(pre_agreement.index) + list(post_agreement.index), fontsize=9)
    plt.grid(alpha=0.2)
    plt.legend(fontsize=10)
    plt.tight_layout()
    # 保存图片
    paris_path = os.path.join(RESULT_DIR, "paris_agreement_final.png")
    plt.savefig(paris_path, dpi=150, facecolor="white")
    plt.close()
    print(f"✅ 协定效果图已保存 → {paris_path}")

    # 分地区评估
    print("\n===== 分地区减排效果 =====")
    continent_mapping = {
        "China":"Asia", "India":"Asia", "United States":"North America", "Germany":"Europe",
        "Brazil":"South America", "South Africa":"Africa", "Australia":"Oceania"
    }
    model_df["Continent"] = model_df["Country"].map(continent_mapping)
    valid_continent = model_df[model_df["Continent"].notna()]
    
    # 存储地区速率（用于后续AI解读）
    continent_rates = {}
    for continent in valid_continent["Continent"].unique():
        cont_data = valid_continent[valid_continent["Continent"] == continent]
        if len(cont_data) > 10:  # 过滤样本量过少的地区
            c_pre = cont_data[cont_data["Year"].between(2010,2015)].groupby("Year")["Per_Capita_GHG"].mean()
            c_post = cont_data[cont_data["Year"].between(2016,2020)].groupby("Year")["Per_Capita_GHG"].mean()
            c_pre_rate = (c_pre.iloc[-1]-c_pre.iloc[0])/5/c_pre.iloc[0]*100
            c_post_rate = (c_post.iloc[-1]-c_post.iloc[0])/5/c_post.iloc[0]*100
            continent_rates[continent] = (c_pre_rate, c_post_rate)
            print(f"{continent}：协定前 {c_pre_rate:.1f}% → 协定后 {c_post_rate:.1f}%")
    
    return pre_rate, post_rate, continent_rates

# 执行评估函数
pre_rate, post_rate, continent_rates = evaluate_paris_agreement()

# ===================== 步骤4：K-Means国家聚类分析 =====================
# 准备聚类数据
cluster_df = model_df[["Country", "Per_Capita_Energy", "Per_Capita_GHG"]].dropna().drop_duplicates(subset=["Country"])
# 归一化特征
mm_scaler = MinMaxScaler()
cluster_features = mm_scaler.fit_transform(cluster_df[["Per_Capita_Energy", "Per_Capita_GHG"]])
# 聚类（3类：低/中/高排放-能耗）
kmeans = KMeans(n_clusters=3, random_state=42)
cluster_df["Cluster"] = kmeans.fit_predict(cluster_features)

# 聚类标签映射
cluster_labels = {0: "低排放-低能耗组", 1: "中排放-中能耗组", 2: "高排放-高能耗组"}
cluster_df["Cluster_Name"] = cluster_df["Cluster"].map(cluster_labels)

# 输出聚类结果
print("\n===== 国家聚类结果 =====")
for cluster in range(3):
    cluster_countries = cluster_df[cluster_df["Cluster"] == cluster]["Country"].unique()[:5]
    print(f"{cluster_labels[cluster]}：{', '.join(cluster_countries)}")

# 可视化聚类结果
plt.figure(figsize=(10, 6), dpi=150)
colors = ["#2A9D8F", "#E9C46A", "#E76F51"]
markers = ["o", "s", "^"]
for cluster in range(3):
    subset = cluster_df[cluster_df["Cluster"] == cluster]
    plt.scatter(subset["Per_Capita_Energy"], subset["Per_Capita_GHG"],
                c=colors[cluster], marker=markers[cluster], s=60, alpha=0.7, label=cluster_labels[cluster])
plt.title("全球国家人均能源消耗-CO₂排放聚类分析（2010-2020）", fontsize=12)
plt.xlabel("人均能源消耗（吨标准煤/人）", fontsize=11)
plt.ylabel("人均CO₂排放（吨/人）", fontsize=11)
plt.grid(alpha=0.2)
plt.legend(fontsize=10)
plt.tight_layout()
# 保存图片
cluster_path = os.path.join(RESULT_DIR, "country_clustering_final.png")
plt.savefig(cluster_path, dpi=150, facecolor="white")
plt.close()
print(f"✅ 聚类图已保存 → {cluster_path}")

# ===================== 步骤5：2021-2030年排放预测 =====================
# 构建时间序列数据
time_df = model_df[["Year", "Per_Capita_GDP", "Per_Capita_Energy", "Per_Capita_GHG"]].groupby("Year").mean().reset_index()
# 训练时间序列回归模型
X_time = scaler.fit_transform(time_df[["Per_Capita_GDP", "Per_Capita_Energy"]])
y_time = time_df["Per_Capita_GHG"].values
time_model = LinearRegression()
time_model.fit(X_time, y_time)

# 生成2021-2030年输入数据（基于历史增速外推）
future_years = np.arange(2021, 2031)
# 计算GDP/能源消耗年均增速
gdp_growth = (time_df["Per_Capita_GDP"].iloc[-1] / time_df["Per_Capita_GDP"].iloc[0]) ** (1/10) - 1
energy_growth = (time_df["Per_Capita_Energy"].iloc[-1] / time_df["Per_Capita_Energy"].iloc[0]) ** (1/10) - 1
# 外推未来值
future_gdp = [time_df["Per_Capita_GDP"].iloc[-1] * (1 + gdp_growth) ** i for i in range(10)]
future_energy = [time_df["Per_Capita_Energy"].iloc[-1] * (1 + energy_growth) ** i for i in range(10)]

# 预测未来排放
future_X = scaler.transform(np.column_stack([future_gdp, future_energy]))
future_ghg = time_model.predict(future_X)

# 可视化历史+预测趋势
plt.figure(figsize=(10, 5), dpi=150)
plt.plot(time_df["Year"], time_df["Per_Capita_GHG"], 
         color="#2A9D8F", linewidth=3, marker="o", label="2010-2020 实际值")
plt.plot(future_years, future_ghg, 
         color="#E76F51", linewidth=3, linestyle="--", marker="s", label="2021-2030 预测值")
plt.title("2010-2030 全球人均CO₂排放历史+预测趋势", fontsize=12)
plt.xlabel("年份", fontsize=11)
plt.ylabel("人均CO₂排放（吨/人）", fontsize=11)
plt.grid(alpha=0.2)
plt.legend(fontsize=10)
plt.tight_layout()
# 保存图片
pred_path = os.path.join(RESULT_DIR, "ghg_prediction_2030.png")
plt.savefig(pred_path, dpi=150, facecolor="white")
plt.close()

# 输出预测结果
print("\n===== 2021-2030年人均CO₂排放预测 =====")
for year, ghg in zip(future_years, future_ghg):
    print(f"{year}年：{ghg:.1f}吨/人")
print(f"✅ 预测图已保存 → {pred_path}")

# ===================== 步骤6：AI提示词与解读生成 =====================
# 核心结果汇总
core_results = f"""
1. 影响因素：人均能源消耗权重{lr_model.coef_[1]:.1f}（正向），人均GDP权重{lr_model.coef_[0]:.1f}（负向），模型R²={r2_score(y, lr_model.predict(X)):.3f}；
2. 巴黎协定效果：全球协定前年均减排{pre_rate:.1f}%，协定后{post_rate:.1f}%，速率提升{abs(post_rate/pre_rate):.1f}倍；
3. 分地区：欧洲减排最快（协定后{continent_rates['Europe'][1]:.1f}%/年），非洲最慢（协定后{continent_rates['Africa'][1]:.1f}%/年）；
4. 聚类结果：全球国家分为低/中/高排放-能耗三组；
5. 预测：2030年全球人均排放预计降至{future_ghg[-1]:.1f}吨/人。
"""

# 生成AI提示词
ai_prompt = f"""
请作为气候变化数据分析专家，基于以下量化结果回答问题：
{core_results}
1. 为什么人均能源消耗是CO₂排放的核心驱动因素？结合全球能源结构现状分析；
2. 巴黎协定后减排速率提升的核心原因是什么？区域差异的底层逻辑是什么？
3. 2030年排放预测结果是否符合1.5℃温控目标？需要哪些政策调整？
4. 国家聚类结果对全球气候政策制定有什么启示？
要求：回答需结合数据，逻辑清晰，每个问题回答不超过200字，语言通俗。
"""

# 模拟AI解读结果
ai_interpretation = f"""
### AI深度解读结果
1. 人均能源消耗是核心驱动因素，因全球60%以上的能源仍来自化石燃料（煤炭/石油/天然气），这类能源燃烧直接释放CO₂；可再生能源占比高的国家（如挪威），能源消耗与排放的关联度显著降低。
2. 巴黎协定后减排提速，核心是各国落地NDC自主贡献目标（如中国双碳、欧盟碳中和）；区域差异源于欧洲能源转型早、资金足，非洲则因经济发展依赖化石能源、气候资金不足导致进展缓慢。
3. 2030年预测排放（{future_ghg[-1]:.1f}吨/人）仍高于1.5℃温控目标对应的1.8吨/人，需加快化石能源退出、提升发展中国家清洁能源补贴。
4. 聚类结果提示政策需“分组施策”：高排放组严控化石能源消费，中排放组优化能源结构，低排放组避免走“先污染后治理”老路。
"""

# 保存AI文件
prompt_path = os.path.join(RESULT_DIR, "ai_analysis_prompt.txt")
with open(prompt_path, "w", encoding="utf-8") as f:
    f.write(ai_prompt)

interpretation_path = os.path.join(RESULT_DIR, "ai_analysis_interpretation.txt")
with open(interpretation_path, "w", encoding="utf-8") as f:
    f.write(ai_interpretation)

print(f"\{prompt_path}")
print(f" {interpretation_path}")

# 交互式仪表盘（适配新版Dash）
# 准备仪表盘数据
cluster_dash = cluster_df[["Country", "Per_Capita_Energy", "Per_Capita_GHG", "Cluster_Name"]].copy()
# 合并历史+预测数据
history_pred = pd.DataFrame({
    "Year": list(time_df["Year"]) + list(future_years),
    "Per_Capita_GHG": list(time_df["Per_Capita_GHG"]) + list(future_ghg),
    "Type": ["历史值"]*11 + ["预测值"]*10
})
# 政策建议字典
policy_suggestions = {
    "高排放-高能耗组": "严控化石能源消费，加速风电/光伏替代，实施碳定价机制（如欧盟碳交易）",
    "中排放-中能耗组": "优化产业结构，淘汰落后产能，提升工业能源利用效率（如中国煤电改造）",
    "低排放-低能耗组": "避免依赖高耗能产业发展，直接引入清洁能源技术（如非洲光伏普及）",
    "全球层面": "建立气候资金转移机制，支持发展中国家能源转型，统一碳核算标准"
}

# 初始化Dash应用
app = dash.Dash(__name__, title="全球人均CO₂排放分析仪表盘")
server = app.server

# 仪表盘布局
app.layout = html.Div([
    # 标题
    html.H1("2010-2030全球人均CO₂排放分析仪表盘", style={"textAlign": "center", "padding": "20px"}),
    
    # 第一行：聚类散点图 + 政策建议
    html.Div([
        # 左侧聚类图
        html.Div([
            dcc.Graph(
                id="cluster-scatter",
                figure=px.scatter(
                    cluster_dash,
                    x="Per_Capita_Energy",
                    y="Per_Capita_GHG",
                    color="Cluster_Name",
                    hover_data=["Country"],
                    title="国家聚类：人均能源消耗 vs 人均CO₂排放",
                    color_discrete_map={
                        "低排放-低能耗组": "#2A9D8F",
                        "中排放-中能耗组": "#E9C46A",
                        "高排放-高能耗组": "#E76F51"
                    },
                    labels={
                        "Per_Capita_Energy": "人均能源消耗（吨标准煤/人）",
                        "Per_Capita_GHG": "人均CO₂排放（吨/人）"
                    }
                ).update_layout(plot_bgcolor="white")
            )
        ], style={"width": "60%", "display": "inline-block", "padding": "10px"}),
        
        # 右侧政策建议
        html.Div([
            html.H3("AI生成政策建议", style={"paddingLeft": "20px"}),
            html.Div([
                html.P(f"{group}：{suggestion}", style={"padding": "10px", "borderBottom": "1px solid #eee"})
                for group, suggestion in policy_suggestions.items()
            ])
        ], style={"width": "35%", "display": "inline-block", "verticalAlign": "top", "padding": "10px"})
    ]),
    
    # 第二行：历史+预测趋势图
    html.Div([
        dcc.Graph(
            id="prediction-trend",
            figure=px.line(
                history_pred,
                x="Year",
                y="Per_Capita_GHG",
                color="Type",
                title="2010-2030年全球人均CO₂排放趋势（历史+预测）",
                color_discrete_map={"历史值": "#2A9D8F", "预测值": "#E76F51"},
                labels={"Per_Capita_GHG": "人均CO₂排放（吨/人）"},
                markers=True
            ).update_layout(
                plot_bgcolor="white",
                shapes=[
                    # 标注巴黎协定年份
                    dict(
                        type="line",
                        x0=2015, x1=2015, y0=0, y1=1, yref="paper",
                        line=dict(color="#666", dash="dash")
                    )
                ],
                annotations=[
                    dict(
                        x=2015, y=0.95, yref="paper",
                        text="巴黎协定签署（2015）",
                        showarrow=True, arrowhead=1
                    )
                ]
            )
        )
    ], style={"padding": "10px"})
])

# 启动仪表盘（适配新版Dash的app.run()）
if __name__ == "__main__":
    print("仪表盘启动中... 请访问 http://127.0.0.1:8050/ 查看")
    app.run(debug=True)  # 替换原app.run_server(debug=True)，适配新版Dash

✅ 数据加载完成 | 有效样本量：1914 | 人均排放范围：0.0~20.5吨/人

===== 人均CO₂排放影响因素权重 =====
     影响因素  权重系数  绝对权重
1  人均能源消耗  1.92  1.92
0   人均GDP -0.09  0.09
模型解释力（R²）：0.536
✅ 权重图已保存 → d:\OneDrive - HKUST (Guangzhou)\桌面\icslab\results\impact_factors_final.png

===== 巴黎协定减排效果 =====
2010-2015（协定前）：年均变化率 -0.5%
2016-2020（协定后）：年均变化率 -1.1%
减排速率提升倍数：2.1倍
✅ 协定效果图已保存 → d:\OneDrive - HKUST (Guangzhou)\桌面\icslab\results\paris_agreement_final.png

===== 分地区减排效果 =====
Oceania：协定前 -2.4% → 协定后 -2.6%
South America：协定前 15.8% → 协定后 -1.1%
Asia：协定前 4.6% → 协定后 2.6%
Europe：协定前 -0.0% → 协定后 -7.5%
Africa：协定前 -2.6% → 协定后 -2.4%
North America：协定前 -3.1% → 协定后 -3.5%

===== 国家聚类结果 =====
低排放-低能耗组：American Samoa, Antigua and Barbuda, Aruba, Australia, Austria
中排放-中能耗组：Afghanistan, Albania, Algeria, Angola, Argentina
高排放-高能耗组：Bahrain, Iceland, Kuwait, Qatar, Singapore
✅ 聚类图已保存 → d:\OneDrive - HKUST (Guangzhou)\桌面\icslab\results\country_clustering_final.png

===== 2021-2030年人均CO₂排放预测 =====
2021年：1.7吨/人
2022年：1.7吨/人
2023年：1.7吨/人
2024年：1.6吨/人
